In [ ]:
import numpy as np
from scipy.signal import welch
from muselsl import stream, list_muses, view

In [ ]:
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter

In [ ]:
import numpy as np
from pylsl import StreamInlet, resolve_byprop
'''
# First, resolve an EEG stream on the lab network
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    # Create a new inlet to read from the stream
    inlet = StreamInlet(streams[0])
    
    while True:
        # Get a new sample from the inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Process the sample (e.g., append it to a buffer)
            print(sample)
else:
    print("No streams found")
'''
import numpy as np
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter

# Function to design a Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Function to apply a bandpass filter to the data
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Function to calculate average power in the beta band
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, nperseg=len(data))
    beta_low, beta_high = 13, 30  # Beta band range in Hz
    idx_beta = np.logical_and(freqs >= beta_low, freqs <= beta_high)
    beta_power = np.mean(psd[idx_beta])
    return beta_power

# Define the sampling frequency (fs) based on your device's specifications
fs = 256  # Hz

# Define a threshold for detecting high concentration from beta power
# This should be calibrated based on the user's baseline beta power levels
high_concentration_threshold = 0.5  # This is an example value

# First, resolve an EEG stream on the lab network
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    # Create a new inlet to read from the stream
    inlet = StreamInlet(streams[0])
    
    # Initialize a buffer to store a certain number of samples
    # For example, to store 5 seconds of data at 256 Hz
    buffer_length = 5 * fs
    eeg_buffer = np.zeros((buffer_length, len(streams[0].channel_format)))
    
    print("Stream found and connected")

    while True:
        # Get a new sample from the inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Process the sample (e.g., append it to a buffer)
            # Shift the existing buffer and append new sample at the end
            eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
            eeg_buffer[-1, :] = np.array(sample)
            
            # Process the buffer only when it's full
            if np.all(eeg_buffer[:, 0] != 0):
                # Preprocess the buffer - apply bandpass filter
                filtered_buffer = bandpass_filter(eeg_buffer, beta_low, beta_high, fs)
                
                # Calculate beta band power
                beta_power = beta_band_power(filtered_buffer, fs)
                
                # Check if beta power exceeds the threshold
                if beta_power > high_concentration_threshold:
                    print("High concentration detected. Triggering game action.")
                    # Here you would trigger a game action (e.g., select or move a piece)
                    # trigger_game_action('move')  # Replace with your game action function

        else:
            print("No new samples found")

We will be using the butterwoth filter

In [ ]:
import numpy as np
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter

# Constants
FS = 256  # Hz, sampling frequency
BETA_LOW, BETA_HIGH = 13, 30  # Beta band range in Hz
BUFFER_LENGTH = 5 * FS  # 5 seconds of data
HIGH_CONCENTRATION_THRESHOLD = 0.5  # Example threshold, calibrate this

# Design the Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

# Calculate the beta band power
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, axis=0)
    idx_beta = np.logical_and(freqs >= BETA_LOW, freqs <= BETA_HIGH)
    beta_power = np.mean(psd[idx_beta], axis=0)
    return beta_power

# Main processing function
def process_eeg_data(inlet):
    # Initialize buffer
    eeg_buffer = np.zeros((BUFFER_LENGTH, inlet.info().channel_count()))
    
    while True:
        # Pull sample from inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Update the buffer
            eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
            eeg_buffer[-1, :] = sample

            # Check if buffer is full
            if np.all(eeg_buffer[:, 0] != 0):
                # Preprocess the buffer - apply bandpass filter
                filtered_data = bandpass_filter(eeg_buffer, BETA_LOW, BETA_HIGH, FS)

                # Calculate beta band power
                beta_power = beta_band_power(filtered_data, FS)

                # Determine high concentration
                high_concentration = beta_power > HIGH_CONCENTRATION_THRESHOLD

                # Output high concentration status
                print(f"High concentration detected: {high_concentration}")
                # Here you would trigger a game action if high concentration is detected
                # trigger_game_action('move') if high_concentration else None

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    inlet = StreamInlet(streams[0])
    print("Stream found and connected")
    process_eeg_data(inlet)  # Start processing EEG data
else:
    print("No streams found")

In [ ]:
import numpy as np
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter
import pyautogui

# Constants for EEG
FS = 256  # Hz
BETA_LOW, BETA_HIGH = 13, 30  # Beta band range in Hz
BUFFER_LENGTH = 5  # Buffer length in seconds
HIGH_CONCENTRATION_THRESHOLD = 0.5  # Example threshold, calibrate this

# Design the Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

# Calculate the beta band power
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, axis=0)
    idx_beta = np.logical_and(freqs >= BETA_LOW, freqs <= BETA_HIGH)
    beta_power = np.mean(psd[idx_beta], axis=0)
    return beta_power

# Main processing function
def process_eeg_data(inlet):
    eeg_buffer = np.zeros((int(FS * BUFFER_LENGTH), 1))
    
    while True:
        # Pull sample from inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Update the buffer with the new sample
            eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
            eeg_buffer[-1, :] = sample

            # Process the buffer when it's full
            if np.all(eeg_buffer[:, 0] != 0):
                # Apply the bandpass filter
                filtered_data = bandpass_filter(eeg_buffer, BETA_LOW, BETA_HIGH, FS)

                # Calculate the beta band power
                beta_power = beta_band_power(filtered_data, FS)

                # Check for high concentration
                if beta_power > HIGH_CONCENTRATION_THRESHOLD:
                    print("High concentration detected. Making the dino jump.")
                    pyautogui.press('space')  # Simulate spacebar press for Dino jump

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    inlet = StreamInlet(streams[0])
    print("Stream found and connected")
    process_eeg_data(inlet)  # Start processing EEG data
else:
    print("No streams found")


In [ ]:
import numpy as np
from pylsl import StreamInlet, resolve_byprop
import pyautogui
from time import sleep

# Constants
FS = 256  # Hz, sampling frequency
BETA_LOW, BETA_HIGH = 13, 30  # Beta band range in Hz
BUFFER_LENGTH = 10 * FS  # 5 seconds of data
HIGH_CONCENTRATION_THRESHOLD = 0.1  # Example threshold, calibrate this

# Design the Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

# Calculate the beta band power
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, axis=0)
    idx_beta = np.logical_and(freqs >= BETA_LOW, freqs <= BETA_HIGH)
    beta_power = np.mean(psd[idx_beta], axis=0)
    return beta_power
def process_eeg_data(inlet):
    # Get the stream info and number of channels
    info = inlet.info()
    n_channels = info.channel_count()
    
    # Initialize buffer to match the number of channels
    eeg_buffer = np.zeros((BUFFER_LENGTH, n_channels))
    
    while True:
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Ensure the incoming sample is a 2D array with the shape (1, n_channels)
            sample = np.array(sample).reshape(1, n_channels)
            
            # Update the buffer
            eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
            eeg_buffer[-1, :] = sample
            
            # Process the buffer when it's full
            if np.all(eeg_buffer[:, 0] != 0):
                filtered_data = bandpass_filter(eeg_buffer, BETA_LOW, BETA_HIGH, FS)
                beta_power = beta_band_power(filtered_data, FS)
                high_concentration = beta_power > HIGH_CONCENTRATION_THRESHOLD
                print("entered buffer loop")

                if high_concentration:
                    print("High concentration detected. Jump!")
                    pyautogui.press('space')  # Simulate spacebar press to jump in Dino game

print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    inlet = StreamInlet(streams[0])
    print("Stream found and connected")
    process_eeg_data(inlet)  # Start processing EEG data
    print(inlet)
else:
    print("No streams found")


In [ ]:
import pyautogui
from time import sleep

# Give yourself a couple of seconds to switch to the game window
print("Switch to the game window. The spacebar will be pressed automatically after 5 seconds...")
sleep(5)

# Press the spacebar - this should make the character in the game jump
pyautogui.press('space')
print("The spacebar was pressed.")

# Run this block to see if the Dino jumps. Make sure the Dino game window is active and in focus.
